In [ ]:
!git clone 'https://github.com/nlp-sakha/sakha-embeddings.git'

Cloning into 'sakha-embeddings'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 55 (delta 0), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (55/55), done.


In [ ]:
%cd sakha-embeddings/

/content/sakha-embeddings


In [ ]:
%mkdir data

In [ ]:
#!python parser/wikitodata.py data/sahwiki-latest-pages-articles.xml.bz2  data/wiki.txt
#!python parser/parser.py kyym -f 0 -t 2000 -o data/corpus.kyym.txt
#!python parser/parser.py edersaas -f 0 -t 100 -o data/corpus.edersaas.txt
#!python parser/parser.py sakhasire -f 0 -t 100 -o data/corpus.sakhasire.txt
#!python parser/parser.py sakhalife_sahalii -f 0 -t 10: -o data/corpus.sakhalife.txt
#!python parser/parser.py iltymen -f 0 -t 2000 -o data/corpus.iltymen.txt

**Самые нужные библиотеки для парсинга сайтов.**

**Requests and BeautifulSoup**

Подробнее о примерах применения на русском языке можно почитать тут:

https://python-scripts.com/requests

https://python-scripts.com/beautifulsoup-html-parsing

Но если у вас хороший английский, то читайте документации

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

**Сайт** **SakhaLife**

In [ ]:
BASE_URL = "https://sakhalife.ru/category/sahalii/"

Копия документа sakha-embeddings/parser/parser_sakhalife_sahalii.py

In [ ]:
def get_html(url):
    r = requests.get(url)
    return r.text


def get_last_page(html):
    soup = BeautifulSoup(html, "lxml")
    last_page = int(soup.find_all('a', class_='page-numbers')[4].text)
    return last_page


def get_page_data(html):
    soup = BeautifulSoup(html, "lxml")
    news_url = soup.find_all('a', class_='new')
    url = []
    for add in news_url:
        try:
            url.append(add.get('href'))
        except:
            url.append('')
    return url


def get_article_content(url):
    page = requests.get(url)

    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "lxml")
        try:
            date = soup.find('div', class_ ='post-date').find('div', class_ ='date').text
            name = soup.find('div', class_ ='post-content').find('h1').text
            content = soup.find('div', class_ ='post-content-inner').text
            return date, name, content
        except Exception as exc:
            print(exc)
    else:
        return None


def parser(output='data/sakhalife_data.txt', print_each=2):
    last_page = get_last_page(get_html(BASE_URL))
    count = 0
    for i in range(1, last_page + 1):
        GEN_URL = BASE_URL+'page/'+str(i)
        html = get_html(GEN_URL)
        urls = get_page_data(html)

        for url in urls:
            text_file = open(output,'a')
            date, name, content = get_article_content(url)
            if content is not None:
                text_file.write('сонун №'+str(count)+'\n')
                text_file.write('дата - ' + date + '\n')
                text_file.write('сонун аата - ' + name+'\n')
                text_file.write(content+'\n')
            text_file.close()
            count += 1
        print(count)
    return count

In [ ]:
parser()

16
32
48
64
80
96
112
128
144
160
176
192
208
224
240
256
272
288
304
320
336
352
368


368

**Сайт** **Эдэр** **Саас**

In [ ]:
BASE_URL = "https://edersaas.ru/"

In [ ]:
def get_html(url):
    r = requests.get(url)
    return r.text


def get_status(url):
    page = requests.get(url)
    return page.status_code


def get_urls(html):
    soup = BeautifulSoup(html, "lxml")
    urls = []
    for url in soup.find('div', class_='navbar').find_all('a')[1:]:
      if len(url.get('href')) > 2:
        urls.append(url.get('href'))
    return urls


def get_page_data(html):
    soup = BeautifulSoup(html, "lxml")
    news_url = soup.find_all('h3', class_="entry-title")
    url = []
    for add in news_url:
        try:
            url.append(add.find('a').get('href'))
        except:
            url.append('')
    return set(url)


def get_article_content(url):
    page = requests.get(url)

    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "lxml")
        try:
            rubrika = soup.find('div', class_='post-content').find('span', class_='cat-links mag-lite-cat-2').text
            date = soup.find('div', class_='post-content').find('time', class_='entry-date published').text.split()[-1]
            name = soup.find('div', class_='post-content').find('h1', class_='entry-title').text.replace('\t', '').replace('\n', '')
            content = soup.find('div', class_='post-content').find('div', class_='entry-content').text.replace('/t', '')
            return rubrika, date, name, content
        except:
            None
    else:
        return None


def parser(output='data/edersaas_data.txt', print_each=2):
    urls = get_urls(get_html(BASE_URL))
    for url in tqdm(urls,):
      i = 1
      count = 1
      while get_status(url+'page/'+str(i)) != 404:
        print(url+'page/'+str(i))
        GEN_URL = url+'page/'+str(i)+'/'
        html = get_html(GEN_URL)
        ssylki = get_page_data(html)
        for ss in ssylki:
          text_file = open(output,'a')
          rubrika, date, name, content = get_article_content(ss)
          if content is not None:
              text_file.write('сонун №'+str(count)+'\n')
              text_file.write('дата - ' + date + '\n')
              text_file.write('рубрика - ' + rubrika + '\n')
              text_file.write('сонун аата - ' + name+'\n')
              text_file.write(content+'\n')
          count = count+1
          text_file.close()
        i = i+1
      print(i,count)
    return count

In [ ]:
parser()

  0%|          | 0/16 [00:00<?, ?it/s]

https://edersaas.ru/category/uopsastuba/page/1


**Сайт** **Кыым**

In [ ]:
BASE_URL = "http://kyym.ru/sonunnar/"

In [ ]:
def get_html(url):
    r = requests.get(url)
    return r.text


def get_last_page(html):
    soup = BeautifulSoup(html, "lxml")
    last_page = int(soup.find_all('p', class_='counter pull-right')[0].text.split()[-1])
    return last_page


def get_page_data(html):
    soup = BeautifulSoup(html, "lxml")
    news_url = soup.find_all('h2', class_='article-title')
    url = []
    for add in news_url:
        try:
            url.append(add.find('a').get('href'))
        except:
            None
    return url


def get_article_content(url):
    page = requests.get("http://kyym.ru"+url)

    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "lxml")
        try:
            title = soup.find_all('h1', class_='article-title')[0].find('a').get('title')
        except:
            title = ''
        try:
            rubrika = soup.find_all('dd', class_='category-name hasTooltip')[0].find('a').text
        except:
            rubrika = ''
        try:
            content = soup.find_all('section', class_='article-content clearfix')[0].text
            date = soup.find_all('dd', class_='published hasTooltip')[0].find('time').get('datetime')
        except:
            date = ''
        return title, rubrika, content, date
    else:
        return None


def parser(output='data/kyym_data2.txt', print_each=2):
    last_page = get_last_page(get_html(BASE_URL))
    count = 2142
    for i in range(102, last_page):
        GEN_URL = BASE_URL[:-1]+'?start='+str(i*21)
        html = get_html(GEN_URL)
        urls = get_page_data(html)
        print(urls)

        for url in urls:
            text_file = open(output,'a')
            title, rubrika, content, date = get_article_content(url)
            if content is not None:
                text_file.write('сонун №'+str(count)+'\n')
                text_file.write('дата - ' + date + '\n')
                text_file.write('рубрика - ' + rubrika + '\n')
                text_file.write('сонун аата - ' + title+'\n')
                text_file.write(content+'\n')
            text_file.close()
            count += 1
        print(count)
    return count

In [ ]:
parser()

**Сайт** **ЯСИА**

In [ ]:
BASE_URL = "https://sakha.ysia.ru/category/s-r-n-sonunnar/"

In [ ]:
r = requests.get(BASE_URL)

In [ ]:
soup = BeautifulSoup(r.text, "lxml")

In [ ]:
soup.find_all(class_='wpdmi-columns')

[<div class="wpdmi-columns wpdmi-columns-layout wpdmi-columns-1 wpdmi-columns-layoutsix wpdmi-columns-normal-space">
 <div class="card-article card-wpdmi-layout card-wpdmi-layoutsix card-wpdmi-normal-space card-wpdmi-theme-dark card-wpdmi-image card-wpdmi-image-top">
 <div class="card-article-body">
 <div class="card-article-content">
 <div class="card-image card-image-wpdminews-normal">
 <div class="card-thumbnail focuspoint" data-focus-x="0" data-focus-y="0"> <!--    data-image-w="690" data-image-h="460" -->
 <div class="card-category"><a href="https://sakha.ysia.ru/category/tya-ha-aajystybata/">Тыа хаһаайыстыбата</a></div> <a href="https://sakha.ysia.ru/saha-sirin-tus-ha-aajystybalara-t-tuttaran-ebii-dohuot-killeriniehtere/">
 <img alt="Саха сирин тус хаһаайыстыбалара үүт туттаран эбии дохуот киллэриниэхтэрэ" loading="lazy" src="https://sakha.ysia.ru/wp-content/uploads/2022/04/yakutskaya-korova-Syuldyukar-690x460.jpg"/>
 </a>
 </div>
 </div>
 <div class="card-text">
 <div class="car

In [ ]:
!pip install selenium

In [ ]:
from time import sleep
from tqdm import tqdm
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [ ]:
browser = Chrome('/content/chromedriver.exe')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


WebDriverException: ignored

In [ ]:
def get_html(url):
    r = requests.get(url)
    return r.text


def get_status(url):
    page = requests.get(url)
    return page.status_code


def get_urls(html):
    soup = BeautifulSoup(html, "lxml")
    urls = []
    for url in soup.find_all('div', class_='card-title'):
        urls.append(url.find_all('a')[-1].get('href'))
    return urls


def get_article_content(url):
    page = requests.get(url)

    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "lxml")
        try:
            content = soup.find('h1').text.replace('\t', '') + soup.find('div', class_="entry-content").text
            return content
        except Exception as exc:
            print(exc)
    else:
        return None


def parser(output='ysia_data.txt', print_each=2):
    urls = get_urls(get_html(BASE_URL))
    for url in urls:
      i = 1
      count = 1
      while get_status(url+'page/'+str(i)) != 404:
        GEN_URL = url+'page/'+str(i)
        html = get_html(GEN_URL)
        ssylki = get_page_data(html)
        for ss in ssylki:
            text_file = open(output,'a')
            content = get_article_content(ss)
            if content is not None:
                text_file.write('сонун №'+str(count)+'\n')
                text_file.write(content+'\n')
            count = count+1
            text_file.close()
        i = i+1
      print(i,count)
    return count

In [ ]:
parser()

**Аартык**

In [ ]:
BASE_URL='https://aartyk.ru/category/sakha/'

In [ ]:
def get_html(url):
    r = requests.get(url)
    return r.text


def get_last_page(html):
    soup = BeautifulSoup(html, "lxml")
    the_last_page = soup.find('div', class_='nav-links').find_all('a', class_='page-numbers')[-2].get('href')
    last_page = int(the_last_page.split('/')[-2])
    return last_page


def get_page_data(html):
    soup = BeautifulSoup(html, "lxml")
    news_url = soup.find_all('h5', class_='title')
    url = []
    for add in news_url:
        try:
            url.append(add.find('a').get('href'))
        except:
            url.append('')
    return url


def get_article_content(url):
    page = requests.get(url)

    if page.status_code == 200:
        text=''
        soup = BeautifulSoup(page.text, "lxml")
        try: 
            for el in soup.find_all('div', attrs={'class':"single-wrapper"}):
              text = text + el.get_text()
            #content = soup.find('div', class_ ='post-content-inner').text
            return text
        except Exception as exc:
            print(exc)
    else:
        return None


def parser(output='data/aartyk_data.txt', print_each=2):
    last_page = get_last_page(get_html(BASE_URL))
    count = 0
    for i in range(1, last_page + 1):
        GEN_URL = BASE_URL+'page/'+str(i)
        html = get_html(GEN_URL)
        urls = get_page_data(html)
        #print(urls)

        for url in urls:
            text_file = open(output,'a')
            content = get_article_content(url)
            if content is not None:
                text_file.write('сонун №'+str(count)+'\n')
                text_file.write(content+'\n')
            text_file.close()
            count += 1
        print(count)
    return count

In [ ]:
parser()